In [4]:
import pandas as pd
import numpy as np
# функции Вовы
import seaborn as sns
from main_bcs_visualization import *
from main_bcs_CDE_targets import *
from datetime import *
from tqdm import tqdm
import inspect
import os
import pandas_profiling
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, recall_score, accuracy_score, r2_score, mean_absolute_error
from catboost import CatBoostRegressor
tscv = TimeSeriesSplit(n_splits=5) 

In [5]:
def data_processing(names_table_):
    names_table = names_table_.copy()
    #     дропаем тех, кто не окончил и не отчислился
    names_table.drop(names_table[(names_table['dropped_before'] == 0)
                                 & (names_table['last_mention-dropped'] == 0)
                                 & (names_table['last_mention-graduated_BCS'] == 0)].index, inplace=True)

    #     дропаем фио и названия факультетов, и колонки кооторые использовали для таргета
    #     тут большой вопрос - надо ли считать количество приказов, академов и т.п., пока не считаем
    cols_to_drop = [
        'ФАМИЛИЯ', 'ИМЯ', 'ОТЧЕСТВО', 'ФАКУЛЬТЕТ_ИД', 'ГРУППА',
        'dropped_before', 'last_mention-dropped', 'last_mention-graduated_BCS',
        'orders_bcs_number', 'exchange_semesters_count', 'gap_year', 'changed_group',
        'last_begin_date'
    ]
    names_table.drop(columns=cols_to_drop, inplace=True)

    #     кастим пол к циферкам
    names_table['ПОЛ'] = names_table['ПОЛ'].apply(lambda x: 0 if x == 'Ж' else 1)
 #     кастим дату к циферкам считая от самого старого чувака
    names_table['ДАТА_РОЖДЕНИЯ'] = names_table['ДАТА_РОЖДЕНИЯ'].apply(fix_two_digit_year)
    names_table['ДАТА_РОЖДЕНИЯ'] = pd.to_datetime(names_table['ДАТА_РОЖДЕНИЯ'], format="%d.%m.%Y")
    #     сделать по месяцам мб
    names_table['first_begin_date'] = pd.to_datetime(names_table['first_begin_date'], format="%Y.%m.%d")

    names_table['years_at_first_date'] = names_table['first_begin_date'] - names_table['ДАТА_РОЖДЕНИЯ']
    names_table['years_at_first_date'] = names_table['years_at_first_date'].apply(lambda x: x.days // 365)
    names_table.drop(columns=['first_begin_date', 'ДАТА_РОЖДЕНИЯ'], inplace=True)
    names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].fillna('Неизвестно', inplace=True)
    words_to_replace = \
        [
            ('п', ''), ('п. ', ''), ('пос ', ''),
            ('село ', ''), ('с. ', ''), ('г. ', ''),
            ('г ', ''), ('город ', ''), ('- ', '-'),
            (' - ', '-'), ('- ', '-')
        ]
    for w in words_to_replace:
        names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'] = names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].apply(
            lambda x: x.lower().replace(w[0], w[1]))
    names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].fillna('Неизвестно', inplace=True)

    towns_dict = defaultdict(int)
    for town in list(names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].value_counts()
                     [names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].value_counts() > 2].index):
        towns_dict[town] = town
    names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'] = names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].apply(lambda x:
                                                                                                    towns_dict[x])
    names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'] = names_table['НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ'].apply(lambda x:
                                                                                                    np.nan if x == 0 else x)

    names_table['ГРАЖДАНСТВО'].fillna('Неизвестно', inplace=True)

    names_table['УСЛОВИЕ_ЗАЧИСЛЕНИЯ_ЛЬГОТА'].fillna('Неизвестно', inplace=True)

    names_table['ФОРМА_ФИНАНСИРОВАНИЯ'].fillna('Неизвестно', inplace=True)

    names_table['ПРЕИМУЩЕСТВЕННО_ПРАВО'].fillna(0, inplace=True)

    names_table['ФАКУЛЬТЕТ'].fillna('Неизвестно', inplace=True)
    names_table = pd.get_dummies(names_table, columns=['ГРАЖДАНСТВО', 'ТИП_ДОКУМЕНТА', 'НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ',
        'ФАКУЛЬТЕТ', 'ФОРМА_ФИНАНСИРОВАНИЯ', 'УСЛОВИЕ_ЗАЧИСЛЕНИЯ_ЛЬГОТА'
    ])
    return names_table

In [132]:
def fit_print(model, X, y, target_name):
    
#just for results' visualization
#in case of categorical target returns ROC-AUC and accuracy
#continious target forces to return r2 and MAE

    split_num = 0
    if target_name=='mark'or target_name=='debt':
        _r2 = []
        _mae = []
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            model.fit(X_train, y_train)
            split_num +=1
            print ('FOLD ', split_num)
            _r2.append(r2_score(model.predict(X_test), y_test))
            _mae.append(mean_absolute_error(model.predict(X_test), y_test))
            print ('  r2_score : ', _r2[-1])
            print ('  MAE : ', _mae[-1])
        print('Mean r2', np.mean(_r2))
        print('Mean MAE', np.mean(_mae))
        return _r2, _mae
    
    else:
        _roc = []
        _acc = []
        _rec = []
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            model.fit(X_train, y_train)

            split_num +=1
            print ('FOLD ', split_num)
        
            _acc.append(model.score(X_test, y_test))
            _roc.append(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
            _rec.append(recall_score(model.predict(X_test), y_test))

            print ('  Accuracy : ', _acc[-1])
            print ('  ROC-AUC : ', _roc[-1])
            print ('  Recall : ', _rec[-1])
        print()
        print('Mean Accuracy', np.mean(_acc))
       # print('Mean ROC-AUC', np.mean(_roc))
        print('Mean Recall', np.mean(_rec))
        return _roc, _acc

In [7]:
#df = df.astype({"a": int, "b": complex})

In [82]:
def changing_data_to_stats(data, to_semester):
    data_copy = data.copy()
    if to_semester<=2:
        return
    data_statistics = from_cols_to_stats(data_copy, to_semester)
    for i in range(1, to_semester+1):
        data_copy.drop(['DEBTS_SEMESTER_'+str(i), 'MARKS_SEMESTER_'+str(i)], axis=1, inplace=True)
    data_copy = pd.merge(data_copy, data_statistics, on = 'ЧЛВК_ИД')
    data_copy.drop_duplicates(keep=False, inplace=True)
    data_copy.drop('ЧЛВК_ИД', axis=1, inplace=True)
    return data_copy

In [9]:
def from_cols_to_stats(data, to_semester):
    
    #creating some basic statistics from all the data marks or debts
    #no matter what target is
    
    if to_semester<=2:
        return
    debts = pd.DataFrame()
    marks = pd.DataFrame()
    for i in range(1,to_semester):
        debts['DEBTS_SEMESTER_'+str(i)] = data['DEBTS_SEMESTER_'+str(i)]
        marks['MARKS_SEMESTER_'+str(i)] = data['MARKS_SEMESTER_'+str(i)]
    data_statistics = pd.DataFrame(data['ЧЛВК_ИД'])
    data_statistics['DEBTS_MEAN'] = np.mean(debts, axis=1)
    data_statistics['DEBTS_STD'] = np.std(debts, axis=1)
    data_statistics['DEBTS_MIN'] = np.min(debts, axis=1)
    data_statistics['DEBTS_MAX'] = np.max(debts, axis=1)
    data_statistics['MARKS_MEAN'] = np.mean(debts, axis=1)
    data_statistics['MARKS_STD'] = np.std(debts, axis=1)
    data_statistics['MARKS_MIN'] = np.min(debts, axis=1)
    data_statistics['MARKS_MAX'] = np.max(debts, axis=1)
    return data_statistics

In [10]:
#   исходный dataframe из таблицы выгрузка_ук_люди.csv, содержащий ид, имена,
    #   пол, гражданство всех студентов, учившихся или учащихся в итмо с 2014 года
    names_table = pd.read_csv('bachelor_integral_profile.csv', encoding='windows-1251')
    date_cols = ['first_begin_date', 'last_begin_date']
    for c in date_cols:
        names_table[c] = pd.to_datetime(names_table[c], format='%Y-%m-%d')
    names_table = add_dynamic_orders_data(names_table, to_semester=3)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  last_expr = (yield from self._async_exec(code_obj, self.user_ns))
100%|██████████| 56548/56548 [00:46<00:00, 1227.43it/s]


In [105]:
df_marks = read_de_data(2)
names_table_marks_target  = get_target_marks(names_table, df_marks, target='mark', to_semester=3)

Almost done: 100%|██████████| 14520/14520 [00:00<00:00, 25439.37it/s]


In [106]:
names_table_marks_target.head()

,ЧЛВК_ИД,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ГРАЖДАНСТВО,ДАТА_РОЖДЕНИЯ,orders_bcs_number,exchange_semesters_count,gap_year,...,DEBTS_SEMESTER_2,DEBTS_SEMESTER_3,MARKS_SEMESTER_1,MARKS_SEMESTER_2,MARKS_SEMESTER_3,MARKS_STD_SEMESTER_1,MARKS_STD_SEMESTER_2,MARKS_STD_SEMESTER_3,dropped_next_semester,target
0,134476,Рыжаков,Максим,Игоревич,М,Российская Федерация,24.03.87,1,0,0,...,0,10,0.00,0.000000,3.666667,0.000000,0.000000,0.902671,0,3.666667
1,135836,Сафонов,Евгений,Александрович,М,Российская Федерация,02.05.86,3,0,0,...,1,3,3.80,3.428571,3.500000,0.400000,0.494872,0.456435,0,3.500000
2,152261,Храбрый,Денис,Валерьевич,М,Российская Федерация,25.11.93,0,0,0,...,0,0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
3,153330,Полушкин,Роман,Сергеевич,М,Российская Федерация,12.07.92,2,0,0,...,2,2,4.25,4.200000,4.500000,1.118034,1.109955,0.866025,0,4.500000
4,153848,Смирнов,Андрей,Павлович,М,Российская Федерация,06.05.93,4,0,0,...,6,9,3.25,3.142857,3.166667,0.433013,0.345736,0.372678,0,3.166667


In [107]:
overall_results={}

### Trial with drop target and marks for the 3rd semester

In [108]:
names_table_marks_target.shape

(12767, 69)

In [109]:
#basic processing for categorical and time features mostly
names_table_marks_target = data_processing(names_table_marks_target)

#statistics have been taken
names_table_marks_target_statistics = changing_data_to_stats(names_table_marks_target, 3)


In [110]:
#outlining targets - binary for dropped_target and names_table_marks_target_column for mark continuous target
dropped_target = names_table_marks_target_statistics.dropped_next_semester
names_table_marks_target_column = names_table_marks_target_statistics.target

In [112]:
#separating target from common dataset
names_table_marks_target_statistics.drop(['target', 'dropped_next_semester', 'MARKS_STD_SEMESTER_3'], axis=1, inplace=True)

In [113]:
#maybe
continious_features = ['sbj_prize_level_0',
       'sbj_prize_level_1', 'sbj_prize_level_2', 'sbj_prize_level_3',
       'sbj_prize_level_4', 'DEBTS_SEMESTER_1', 'DEBTS_SEMESTER_2', 'orders_bcs_number', 'exchange_semesters_count',
       'gap_year', 'changed_group', 'dropped_before',   'ege_russian_points', 'ege_maths_points', 'ege_informatics_points',
       'ege_physics_points', 'ege_foreign_lang_points',
       'ege_social_studies_points', 'ege_chemistry_points',
       'ege_biology_points', 'uni_russian_points', 'uni_maths_points',
       'uni_informatics_points', 'uni_physics_points',
       'uni_foreign_lang_points', 'uni_social_studies_points',
       'uni_chemistry_points']


In [114]:
nan_cols = []
for col in names_table_marks_target_statistics.columns:
    if names_table_marks_target_statistics[col].isna().value_counts().shape[0] > 1:
        nan_cols.append(col)
        print(col)
for col in nan_cols:
    names_table_marks_target_statistics[col] = names_table_marks_target_statistics[col].fillna(
        names_table_marks_target_statistics[col].mean())

ege_russian_points
ege_maths_points
ege_informatics_points
ege_physics_points
ege_foreign_lang_points
ege_social_studies_points
ege_chemistry_points
ege_biology_points
uni_russian_points
uni_maths_points
uni_informatics_points
uni_physics_points
uni_foreign_lang_points
uni_social_studies_points
uni_chemistry_points


In [115]:
def pipeline_model(model, X, y, target_name):
    
    #learning, returns fitted model and some metrics
    
    if target_name=='mark' or target_name=='debt':
        tscv_test = TimeSeriesSplit(n_splits=2)
        for train_index, test_index in tscv_test.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            fitted_model = model.fit(X_train, y_train)
            prediction = fitted_model.predict(X_test)
            print(prediction[:10])
            print(y[:10])
            acc = mean_absolute_error(prediction, y_test)
    else:
        tscv_test = TimeSeriesSplit(n_splits=2)
        for train_index, test_index in tscv_test.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            fitted_model = model.fit(X_train, y_train)
            prediction = fitted_model.predict(X_test)
            print(prediction[:10])
            print(y[:10])
            acc = accuracy_score(prediction, y_test)
    return fitted_model, acc

In [116]:
y = names_table_marks_target_column.values
X = names_table_marks_target_statistics.values

In [117]:
names_table_marks_target_statistics.head()

,ПОЛ,ege_russian_points,ege_maths_points,ege_informatics_points,ege_physics_points,ege_foreign_lang_points,ege_social_studies_points,ege_chemistry_points,ege_biology_points,uni_russian_points,...,УСЛОВИЕ_ЗАЧИСЛЕНИЯ_ЛЬГОТА_по государственной линии,УСЛОВИЕ_ЗАЧИСЛЕНИЯ_ЛЬГОТА_по общему конкурсу,DEBTS_MEAN,DEBTS_STD,DEBTS_MIN,DEBTS_MAX,MARKS_MEAN,MARKS_STD,MARKS_MIN,MARKS_MAX
0,1,83.509119,77.985612,81.162763,76.104167,84.021739,71.54717,78.102564,83.25,80.118919,...,0,1,0.0,0.0,0,0,0.0,0.0,0,0
1,1,83.509119,77.985612,81.162763,76.104167,84.021739,71.54717,78.102564,83.25,80.118919,...,0,0,4.5,3.5,1,8,4.5,3.5,1,8
2,1,83.509119,77.985612,81.162763,76.104167,84.021739,71.54717,78.102564,83.25,80.118919,...,0,1,0.0,0.0,0,0,0.0,0.0,0,0
3,1,83.509119,77.985612,81.162763,76.104167,84.021739,71.54717,78.102564,83.25,80.118919,...,0,1,1.0,1.0,0,2,1.0,1.0,0,2
4,1,83.509119,77.985612,81.162763,76.104167,84.021739,71.54717,78.102564,83.25,80.118919,...,0,0,3.5,2.5,1,6,3.5,2.5,1,6


In [118]:
svr = SVR()
svr_r2, svr_mae = fit_print(svr, X, y, 'mark')

/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


FOLD  1
  r2_score :  -0.6247849812066746
  MAE :  1.281242924560125


/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


FOLD  2
  r2_score :  -0.3578848735439202
  MAE :  1.207840426250829


/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


FOLD  3
  r2_score :  -0.39886549305083774
  MAE :  1.130186855901932


/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


FOLD  4
  r2_score :  -0.5008526622702858
  MAE :  1.259731554800866


/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


FOLD  5
  r2_score :  -0.2733832501315463
  MAE :  1.1924680350400674
Mean r2 -0.43115425204065294
Mean MAE 1.214293959310764


In [119]:
overall_results['r2 score for SVM on 3rd semester']=svr_r2
overall_results['MAE for SVM on 3rd semester']=svr_mae

In [120]:
svr_fitted, svr_mae = pipeline_model(svr, X, y, 'mark')

/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[2.36471542 3.00624036 4.82950914 3.43548493 2.29082274 2.86664373
 3.38913766 1.85706503 1.02711529 0.70652875]
[3.66666667 3.5        0.         4.5        3.16666667 0.
 3.         0.         0.         3.2       ]


/home/kapitan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[-0.32825985  0.38469186  0.35904165  0.38469186  0.1239833  -0.08748612
  0.36072534  0.11528194  0.11433083  1.28026419]
[3.66666667 3.5        0.         4.5        3.16666667 0.
 3.         0.         0.         3.2       ]


In [121]:
svr_mae

1.2334666721378371

In [122]:
catb = CatBoostRegressor(iterations=500)
catb_r2, catb_mae = fit_print(catb, X, y, 'mark')

0:	learn: 1.9305913	total: 2ms	remaining: 996ms
1:	learn: 1.9057501	total: 6.88ms	remaining: 1.71s
2:	learn: 1.8842809	total: 8.43ms	remaining: 1.4s
3:	learn: 1.8640935	total: 10.8ms	remaining: 1.34s
4:	learn: 1.8468030	total: 12.1ms	remaining: 1.2s
5:	learn: 1.8272718	total: 13.5ms	remaining: 1.11s
6:	learn: 1.8061699	total: 14.9ms	remaining: 1.05s
7:	learn: 1.7874548	total: 16.3ms	remaining: 1s
8:	learn: 1.7701614	total: 22ms	remaining: 1.2s
9:	learn: 1.7557330	total: 24.5ms	remaining: 1.2s
10:	learn: 1.7380114	total: 27.5ms	remaining: 1.22s
11:	learn: 1.7211634	total: 30.6ms	remaining: 1.25s
12:	learn: 1.7086522	total: 32.1ms	remaining: 1.2s
13:	learn: 1.6976376	total: 37.4ms	remaining: 1.3s
14:	learn: 1.6828517	total: 39.3ms	remaining: 1.27s
15:	learn: 1.6686815	total: 41ms	remaining: 1.24s
16:	learn: 1.6581159	total: 42.3ms	remaining: 1.2s
17:	learn: 1.6468749	total: 43.7ms	remaining: 1.17s
18:	learn: 1.6353455	total: 45ms	remaining: 1.14s
19:	learn: 1.6255654	total: 46.7ms	remain

185:	learn: 1.2849947	total: 385ms	remaining: 649ms
186:	learn: 1.2848332	total: 386ms	remaining: 647ms
187:	learn: 1.2847423	total: 388ms	remaining: 644ms
188:	learn: 1.2842749	total: 389ms	remaining: 641ms
189:	learn: 1.2841479	total: 391ms	remaining: 637ms
190:	learn: 1.2838783	total: 392ms	remaining: 634ms
191:	learn: 1.2829655	total: 393ms	remaining: 631ms
192:	learn: 1.2828422	total: 399ms	remaining: 634ms
193:	learn: 1.2821124	total: 400ms	remaining: 632ms
194:	learn: 1.2819360	total: 402ms	remaining: 629ms
195:	learn: 1.2816633	total: 404ms	remaining: 627ms
196:	learn: 1.2803734	total: 407ms	remaining: 626ms
197:	learn: 1.2796782	total: 408ms	remaining: 623ms
198:	learn: 1.2795188	total: 413ms	remaining: 624ms
199:	learn: 1.2786896	total: 414ms	remaining: 621ms
200:	learn: 1.2785711	total: 415ms	remaining: 618ms
201:	learn: 1.2780200	total: 417ms	remaining: 615ms
202:	learn: 1.2779456	total: 418ms	remaining: 611ms
203:	learn: 1.2773845	total: 419ms	remaining: 608ms
204:	learn: 

399:	learn: 1.1821789	total: 688ms	remaining: 172ms
400:	learn: 1.1821166	total: 690ms	remaining: 170ms
401:	learn: 1.1816598	total: 691ms	remaining: 169ms
402:	learn: 1.1808661	total: 696ms	remaining: 168ms
403:	learn: 1.1803174	total: 698ms	remaining: 166ms
404:	learn: 1.1798245	total: 700ms	remaining: 164ms
405:	learn: 1.1793505	total: 702ms	remaining: 162ms
406:	learn: 1.1786799	total: 703ms	remaining: 161ms
407:	learn: 1.1777118	total: 705ms	remaining: 159ms
408:	learn: 1.1771662	total: 707ms	remaining: 157ms
409:	learn: 1.1770009	total: 709ms	remaining: 156ms
410:	learn: 1.1760946	total: 710ms	remaining: 154ms
411:	learn: 1.1759615	total: 712ms	remaining: 152ms
412:	learn: 1.1759058	total: 713ms	remaining: 150ms
413:	learn: 1.1752023	total: 715ms	remaining: 149ms
414:	learn: 1.1748127	total: 716ms	remaining: 147ms
415:	learn: 1.1742076	total: 718ms	remaining: 145ms
416:	learn: 1.1739234	total: 720ms	remaining: 143ms
417:	learn: 1.1736341	total: 721ms	remaining: 141ms
418:	learn: 

120:	learn: 1.2936466	total: 281ms	remaining: 879ms
121:	learn: 1.2933482	total: 283ms	remaining: 877ms
122:	learn: 1.2929029	total: 286ms	remaining: 876ms
123:	learn: 1.2923134	total: 288ms	remaining: 875ms
124:	learn: 1.2921025	total: 291ms	remaining: 874ms
125:	learn: 1.2918308	total: 294ms	remaining: 873ms
126:	learn: 1.2906247	total: 296ms	remaining: 869ms
127:	learn: 1.2902457	total: 299ms	remaining: 868ms
128:	learn: 1.2899888	total: 301ms	remaining: 864ms
129:	learn: 1.2894268	total: 302ms	remaining: 861ms
130:	learn: 1.2887571	total: 304ms	remaining: 857ms
131:	learn: 1.2879483	total: 306ms	remaining: 853ms
132:	learn: 1.2872503	total: 308ms	remaining: 850ms
133:	learn: 1.2870235	total: 310ms	remaining: 846ms
134:	learn: 1.2863665	total: 311ms	remaining: 842ms
135:	learn: 1.2860935	total: 313ms	remaining: 838ms
136:	learn: 1.2855185	total: 315ms	remaining: 835ms
137:	learn: 1.2850275	total: 317ms	remaining: 832ms
138:	learn: 1.2844591	total: 319ms	remaining: 828ms
139:	learn: 

292:	learn: 1.2332257	total: 664ms	remaining: 469ms
293:	learn: 1.2330040	total: 669ms	remaining: 469ms
294:	learn: 1.2324990	total: 672ms	remaining: 467ms
295:	learn: 1.2318118	total: 673ms	remaining: 464ms
296:	learn: 1.2313687	total: 675ms	remaining: 461ms
297:	learn: 1.2308318	total: 683ms	remaining: 463ms
298:	learn: 1.2307451	total: 687ms	remaining: 462ms
299:	learn: 1.2301866	total: 689ms	remaining: 459ms
300:	learn: 1.2300895	total: 691ms	remaining: 457ms
301:	learn: 1.2299097	total: 693ms	remaining: 454ms
302:	learn: 1.2296791	total: 696ms	remaining: 452ms
303:	learn: 1.2295867	total: 699ms	remaining: 451ms
304:	learn: 1.2295268	total: 701ms	remaining: 448ms
305:	learn: 1.2290150	total: 703ms	remaining: 446ms
306:	learn: 1.2284481	total: 705ms	remaining: 443ms
307:	learn: 1.2283174	total: 707ms	remaining: 440ms
308:	learn: 1.2282559	total: 709ms	remaining: 438ms
309:	learn: 1.2281559	total: 711ms	remaining: 436ms
310:	learn: 1.2276195	total: 714ms	remaining: 434ms
311:	learn: 

...

0:	learn: 1.9309430	total: 2.83ms	remaining: 1.42s
1:	learn: 1.9062575	total: 5.71ms	remaining: 1.42s
2:	learn: 1.8802900	total: 9.97ms	remaining: 1.65s
3:	learn: 1.8564004	total: 12.6ms	remaining: 1.56s
4:	learn: 1.8336138	total: 15.2ms	remaining: 1.51s
5:	learn: 1.8112193	total: 18ms	remaining: 1.48s
6:	learn: 1.7913315	total: 20.6ms	remaining: 1.45s
7:	learn: 1.7716951	total: 23.2ms	remaining: 1.42s
8:	learn: 1.7539534	total: 25.8ms	remaining: 1.41s
9:	learn: 1.7350454	total: 28.6ms	remaining: 1.4s
10:	learn: 1.7162229	total: 31.4ms	remaining: 1.39s
11:	learn: 1.6990639	total: 34.1ms	remaining: 1.39s
12:	learn: 1.6823480	total: 36.8ms	remaining: 1.38s
13:	learn: 1.6652493	total: 39.6ms	remaining: 1.37s
14:	learn: 1.6511207	total: 42.2ms	remaining: 1.36s
15:	learn: 1.6356596	total: 44.9ms	remaining: 1.36s
16:	learn: 1.6213591	total: 47.8ms	remaining: 1.36s
17:	learn: 1.6106968	total: 50.4ms	remaining: 1.35s
18:	learn: 1.6006155	total: 52.9ms	remaining: 1.34s
19:	learn: 1.5896442	tota

211:	learn: 1.2779126	total: 588ms	remaining: 799ms
212:	learn: 1.2778185	total: 593ms	remaining: 798ms
213:	learn: 1.2776472	total: 597ms	remaining: 797ms
214:	learn: 1.2774077	total: 600ms	remaining: 796ms
215:	learn: 1.2771503	total: 605ms	remaining: 795ms
216:	learn: 1.2770329	total: 609ms	remaining: 794ms
217:	learn: 1.2768140	total: 612ms	remaining: 791ms
218:	learn: 1.2762716	total: 614ms	remaining: 788ms
219:	learn: 1.2756148	total: 618ms	remaining: 787ms
220:	learn: 1.2751161	total: 621ms	remaining: 784ms
221:	learn: 1.2748789	total: 624ms	remaining: 781ms
222:	learn: 1.2748299	total: 626ms	remaining: 778ms
223:	learn: 1.2746555	total: 629ms	remaining: 775ms
224:	learn: 1.2745368	total: 632ms	remaining: 772ms
225:	learn: 1.2742921	total: 634ms	remaining: 769ms
226:	learn: 1.2740291	total: 637ms	remaining: 766ms
227:	learn: 1.2738890	total: 639ms	remaining: 762ms
228:	learn: 1.2735972	total: 642ms	remaining: 759ms
229:	learn: 1.2734962	total: 644ms	remaining: 756ms
230:	learn: 

428:	learn: 1.2250833	total: 1.17s	remaining: 194ms
429:	learn: 1.2250611	total: 1.18s	remaining: 192ms
430:	learn: 1.2248475	total: 1.18s	remaining: 190ms
431:	learn: 1.2244470	total: 1.19s	remaining: 187ms
432:	learn: 1.2240508	total: 1.19s	remaining: 184ms
433:	learn: 1.2240256	total: 1.19s	remaining: 182ms
434:	learn: 1.2237545	total: 1.2s	remaining: 179ms
435:	learn: 1.2234859	total: 1.2s	remaining: 176ms
436:	learn: 1.2230564	total: 1.2s	remaining: 173ms
437:	learn: 1.2230258	total: 1.2s	remaining: 170ms
438:	learn: 1.2225013	total: 1.21s	remaining: 168ms
439:	learn: 1.2221155	total: 1.21s	remaining: 165ms
440:	learn: 1.2218293	total: 1.21s	remaining: 162ms
441:	learn: 1.2216703	total: 1.22s	remaining: 159ms
442:	learn: 1.2214612	total: 1.22s	remaining: 157ms
443:	learn: 1.2212231	total: 1.22s	remaining: 154ms
444:	learn: 1.2211573	total: 1.22s	remaining: 151ms
445:	learn: 1.2208580	total: 1.23s	remaining: 148ms
446:	learn: 1.2207765	total: 1.23s	remaining: 146ms
447:	learn: 1.22

140:	learn: 1.2939180	total: 397ms	remaining: 1.01s
141:	learn: 1.2936264	total: 400ms	remaining: 1.01s
142:	learn: 1.2933178	total: 405ms	remaining: 1.01s
143:	learn: 1.2929926	total: 409ms	remaining: 1.01s
144:	learn: 1.2926627	total: 413ms	remaining: 1.01s
145:	learn: 1.2923779	total: 416ms	remaining: 1.01s
146:	learn: 1.2922166	total: 420ms	remaining: 1.01s
147:	learn: 1.2918943	total: 423ms	remaining: 1.01s
148:	learn: 1.2913644	total: 427ms	remaining: 1s
149:	learn: 1.2911034	total: 429ms	remaining: 1s
150:	learn: 1.2908848	total: 432ms	remaining: 998ms
151:	learn: 1.2906894	total: 434ms	remaining: 995ms
152:	learn: 1.2898794	total: 437ms	remaining: 991ms
153:	learn: 1.2897332	total: 440ms	remaining: 988ms
154:	learn: 1.2892908	total: 442ms	remaining: 985ms
155:	learn: 1.2888735	total: 445ms	remaining: 981ms
156:	learn: 1.2886453	total: 448ms	remaining: 978ms
157:	learn: 1.2884754	total: 451ms	remaining: 975ms
158:	learn: 1.2880097	total: 453ms	remaining: 972ms
159:	learn: 1.2878

354:	learn: 1.2460605	total: 980ms	remaining: 400ms
355:	learn: 1.2460334	total: 984ms	remaining: 398ms
356:	learn: 1.2458643	total: 988ms	remaining: 396ms
357:	learn: 1.2455065	total: 992ms	remaining: 393ms
358:	learn: 1.2454764	total: 995ms	remaining: 391ms
359:	learn: 1.2454519	total: 998ms	remaining: 388ms
360:	learn: 1.2452054	total: 1s	remaining: 386ms
361:	learn: 1.2451116	total: 1.01s	remaining: 384ms
362:	learn: 1.2450799	total: 1.01s	remaining: 381ms
363:	learn: 1.2448515	total: 1.01s	remaining: 379ms
364:	learn: 1.2446242	total: 1.02s	remaining: 376ms
365:	learn: 1.2443267	total: 1.02s	remaining: 374ms
366:	learn: 1.2442993	total: 1.02s	remaining: 371ms
367:	learn: 1.2441573	total: 1.03s	remaining: 368ms
368:	learn: 1.2439006	total: 1.03s	remaining: 365ms
369:	learn: 1.2435600	total: 1.03s	remaining: 362ms
370:	learn: 1.2435306	total: 1.03s	remaining: 360ms
371:	learn: 1.2432827	total: 1.04s	remaining: 357ms
372:	learn: 1.2432537	total: 1.04s	remaining: 354ms
373:	learn: 1.2

In [123]:
overall_results['r2 score for CatBoost on 3rd semester']=catb_r2
overall_results['MAE for CatBoost on 3rd semester']=catb_mae

In [124]:
catb_fitted, catb_mae = pipeline_model(catb, X, y, 'mark')

0:	learn: 1.9348559	total: 4.6ms	remaining: 2.29s
1:	learn: 1.9112798	total: 7.17ms	remaining: 1.78s
2:	learn: 1.8884249	total: 9.12ms	remaining: 1.51s
3:	learn: 1.8639476	total: 11ms	remaining: 1.36s
4:	learn: 1.8387088	total: 12.9ms	remaining: 1.28s
5:	learn: 1.8173237	total: 16ms	remaining: 1.32s
6:	learn: 1.7967345	total: 18ms	remaining: 1.27s
7:	learn: 1.7781613	total: 19.9ms	remaining: 1.22s
8:	learn: 1.7581882	total: 21.8ms	remaining: 1.19s
9:	learn: 1.7385325	total: 23.6ms	remaining: 1.16s
10:	learn: 1.7190354	total: 25.5ms	remaining: 1.13s
11:	learn: 1.7034799	total: 27.3ms	remaining: 1.11s
12:	learn: 1.6855880	total: 29.2ms	remaining: 1.09s
13:	learn: 1.6713502	total: 31.7ms	remaining: 1.1s
14:	learn: 1.6554285	total: 33.4ms	remaining: 1.08s
15:	learn: 1.6450891	total: 35.2ms	remaining: 1.06s
16:	learn: 1.6332067	total: 37ms	remaining: 1.05s
17:	learn: 1.6208446	total: 38.8ms	remaining: 1.04s
18:	learn: 1.6086815	total: 40.6ms	remaining: 1.03s
19:	learn: 1.5960334	total: 42.5

200:	learn: 1.2611248	total: 391ms	remaining: 582ms
201:	learn: 1.2609428	total: 395ms	remaining: 582ms
202:	learn: 1.2606833	total: 398ms	remaining: 582ms
203:	learn: 1.2597815	total: 400ms	remaining: 581ms
204:	learn: 1.2592550	total: 402ms	remaining: 578ms
205:	learn: 1.2591608	total: 404ms	remaining: 576ms
206:	learn: 1.2590796	total: 405ms	remaining: 574ms
207:	learn: 1.2588820	total: 407ms	remaining: 572ms
208:	learn: 1.2587231	total: 409ms	remaining: 569ms
209:	learn: 1.2583914	total: 411ms	remaining: 567ms
210:	learn: 1.2583622	total: 413ms	remaining: 565ms
211:	learn: 1.2582774	total: 415ms	remaining: 563ms
212:	learn: 1.2581442	total: 417ms	remaining: 562ms
213:	learn: 1.2579791	total: 419ms	remaining: 560ms
214:	learn: 1.2578226	total: 425ms	remaining: 564ms
215:	learn: 1.2576788	total: 434ms	remaining: 570ms
216:	learn: 1.2574130	total: 436ms	remaining: 568ms
217:	learn: 1.2571900	total: 440ms	remaining: 569ms
218:	learn: 1.2569757	total: 443ms	remaining: 568ms
219:	learn: 

400:	learn: 1.2005632	total: 781ms	remaining: 193ms
401:	learn: 1.2005035	total: 784ms	remaining: 191ms
402:	learn: 1.2004468	total: 786ms	remaining: 189ms
403:	learn: 1.2000514	total: 788ms	remaining: 187ms
404:	learn: 1.1998031	total: 792ms	remaining: 186ms
405:	learn: 1.1997524	total: 795ms	remaining: 184ms
406:	learn: 1.1994366	total: 797ms	remaining: 182ms
407:	learn: 1.1994031	total: 800ms	remaining: 180ms
408:	learn: 1.1993411	total: 802ms	remaining: 178ms
409:	learn: 1.1993094	total: 804ms	remaining: 176ms
410:	learn: 1.1991519	total: 807ms	remaining: 175ms
411:	learn: 1.1985946	total: 808ms	remaining: 173ms
412:	learn: 1.1978520	total: 810ms	remaining: 171ms
413:	learn: 1.1977987	total: 812ms	remaining: 169ms
414:	learn: 1.1974848	total: 814ms	remaining: 167ms
415:	learn: 1.1974507	total: 816ms	remaining: 165ms
416:	learn: 1.1974178	total: 817ms	remaining: 163ms
417:	learn: 1.1973580	total: 819ms	remaining: 161ms
418:	learn: 1.1971565	total: 821ms	remaining: 159ms
419:	learn: 

75:	learn: 1.3428970	total: 216ms	remaining: 1.2s
76:	learn: 1.3421656	total: 222ms	remaining: 1.22s
77:	learn: 1.3413823	total: 225ms	remaining: 1.21s
78:	learn: 1.3400607	total: 227ms	remaining: 1.21s
79:	learn: 1.3393293	total: 239ms	remaining: 1.26s
80:	learn: 1.3382077	total: 244ms	remaining: 1.26s
81:	learn: 1.3365344	total: 248ms	remaining: 1.26s
82:	learn: 1.3358571	total: 250ms	remaining: 1.26s
83:	learn: 1.3348620	total: 253ms	remaining: 1.25s
84:	learn: 1.3337748	total: 256ms	remaining: 1.25s
85:	learn: 1.3321444	total: 259ms	remaining: 1.25s
86:	learn: 1.3316735	total: 262ms	remaining: 1.24s
87:	learn: 1.3305967	total: 266ms	remaining: 1.25s
88:	learn: 1.3297926	total: 269ms	remaining: 1.24s
89:	learn: 1.3290886	total: 272ms	remaining: 1.24s
90:	learn: 1.3280416	total: 274ms	remaining: 1.23s
91:	learn: 1.3267111	total: 278ms	remaining: 1.23s
92:	learn: 1.3257762	total: 282ms	remaining: 1.23s
93:	learn: 1.3250184	total: 285ms	remaining: 1.23s
94:	learn: 1.3241903	total: 287m

272:	learn: 1.2616481	total: 805ms	remaining: 669ms
273:	learn: 1.2609075	total: 809ms	remaining: 667ms
274:	learn: 1.2608274	total: 811ms	remaining: 664ms
275:	learn: 1.2607174	total: 814ms	remaining: 661ms
276:	learn: 1.2605720	total: 816ms	remaining: 657ms
277:	learn: 1.2603979	total: 819ms	remaining: 654ms
278:	learn: 1.2601359	total: 823ms	remaining: 652ms
279:	learn: 1.2600438	total: 828ms	remaining: 650ms
280:	learn: 1.2596450	total: 832ms	remaining: 648ms
281:	learn: 1.2595893	total: 836ms	remaining: 647ms
282:	learn: 1.2590544	total: 841ms	remaining: 645ms
283:	learn: 1.2588575	total: 844ms	remaining: 642ms
284:	learn: 1.2587226	total: 846ms	remaining: 638ms
285:	learn: 1.2585894	total: 849ms	remaining: 635ms
286:	learn: 1.2584321	total: 851ms	remaining: 632ms
287:	learn: 1.2580569	total: 855ms	remaining: 629ms
288:	learn: 1.2580082	total: 857ms	remaining: 626ms
289:	learn: 1.2579545	total: 860ms	remaining: 623ms
290:	learn: 1.2578988	total: 862ms	remaining: 619ms
291:	learn: 

470:	learn: 1.2162980	total: 1.39s	remaining: 85.6ms
471:	learn: 1.2162257	total: 1.39s	remaining: 82.7ms
472:	learn: 1.2159365	total: 1.4s	remaining: 79.8ms
473:	learn: 1.2156578	total: 1.41s	remaining: 77.1ms
474:	learn: 1.2153513	total: 1.41s	remaining: 74.3ms
475:	learn: 1.2153064	total: 1.42s	remaining: 71.6ms
476:	learn: 1.2150336	total: 1.42s	remaining: 68.6ms
477:	learn: 1.2146753	total: 1.43s	remaining: 65.6ms
478:	learn: 1.2146494	total: 1.43s	remaining: 62.8ms
479:	learn: 1.2146001	total: 1.44s	remaining: 59.9ms
480:	learn: 1.2145662	total: 1.44s	remaining: 56.8ms
481:	learn: 1.2143357	total: 1.44s	remaining: 53.9ms
482:	learn: 1.2139731	total: 1.45s	remaining: 51ms
483:	learn: 1.2137711	total: 1.45s	remaining: 48ms
484:	learn: 1.2136063	total: 1.46s	remaining: 45ms
485:	learn: 1.2132495	total: 1.46s	remaining: 42ms
486:	learn: 1.2129383	total: 1.46s	remaining: 39ms
487:	learn: 1.2126430	total: 1.47s	remaining: 36.1ms
488:	learn: 1.2123877	total: 1.47s	remaining: 33.1ms
489:

In [125]:
catb_importances = catb_fitted.feature_importances_

In [126]:
catb_imp = [
        [i, round(j, 4)] for i, j in 
        zip(names_table_marks_target_statistics.columns, catb_fitted.feature_importances_.flatten())
            ]
catb_imp = pd.DataFrame(sorted(catb_imp, key = lambda x: x[1], reverse = True)).head(20), 

In [127]:
catb_imp

(                                                    0        1
 0                                MARKS_STD_SEMESTER_2  25.4104
 1                                 years_at_first_date  21.1050
 2   ТИП_ДОКУМЕНТА_Аттестат о среднем общем образов...  13.6437
 3         НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ_санкт-етербург   6.0950
 4        ФОРМА_ФИНАНСИРОВАНИЯ_Из федерального бюджета   4.9706
 5                                MARKS_STD_SEMESTER_1   4.6325
 6        УСЛОВИЕ_ЗАЧИСЛЕНИЯ_ЛЬГОТА_по общему конкурсу   2.7369
 7                                    ege_maths_points   1.5354
 8                                          MARKS_MEAN   1.5186
 9                                  ege_russian_points   1.2622
 10               УСЛОВИЕ_ЗАЧИСЛЕНИЯ_ЛЬГОТА_Неизвестно   1.2609
 11                                 ФАКУЛЬТЕТ_ФПИ и КТ   0.8776
 12            НАСЕЛЕННЫЙ_ПУНКТ_ОБРАЗОВАНИЯ_неизвестно   0.8610
 13                                         DEBTS_MEAN   0.8592
 14                                     

In [128]:
y = dropped_target.values
X = names_table_marks_target_statistics.values

In [130]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [133]:
randf = RandomForestClassifier()
randf_roc, randf_acc = fit_print(randf, X, y, 'binary')

/home/kapitan/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


FOLD  1
  Accuracy :  0.9443651925820257
  ROC-AUC :  0.7443625854770433
  Recall :  0.4
FOLD  2
  Accuracy :  0.920114122681883
  ROC-AUC :  0.7529977603994567
  Recall :  0.3793103448275862
FOLD  3
  Accuracy :  0.9614835948644793
  ROC-AUC :  0.7275200358056981
  Recall :  0.4666666666666667
FOLD  4
  Accuracy :  0.956490727532097
  ROC-AUC :  0.7331445022288261
  Recall :  0.3333333333333333
FOLD  5
  Accuracy :  0.9379457917261056
  ROC-AUC :  0.7283539008193501
  Recall :  0.42857142857142855

Mean Accuracy 0.9440798858773182
Mean Recall 0.40157635467980296


In [134]:
overall_results['ROC-AUC for Random Forest on 3rd semester']=randf_roc
overall_results['Accuracy for Random Forest on 3rd semester']=randf_acc

In [135]:
randf_fitted, randf_acc = pipeline_model(randf, X, y, 'binary')

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]


In [138]:
randf_imp = randf_fitted.feature_importances_
randf_imp = [
        [i, round(j, 4)] for i, j in 
        zip(names_table_marks_target_statistics.columns, randf_fitted.feature_importances_.flatten())
            ]
randf_imp = pd.DataFrame(sorted(randf_imp, key = lambda x: x[1], reverse = True)).head(20), 

In [139]:
randf_imp

(                                                    0       1
 0                                 years_at_first_date  0.0942
 1                                MARKS_STD_SEMESTER_1  0.0859
 2                                MARKS_STD_SEMESTER_2  0.0724
 3                                          MARKS_MEAN  0.0454
 4                                           DEBTS_MAX  0.0432
 5                                    ege_maths_points  0.0316
 6                                          DEBTS_MEAN  0.0303
 7   ТИП_ДОКУМЕНТА_Аттестат о среднем общем образов...  0.0298
 8                                  ege_russian_points  0.0295
 9                                           MARKS_MIN  0.0265
 10                                          MARKS_MAX  0.0255
 11                             ege_informatics_points  0.0230
 12                                          DEBTS_STD  0.0230
 13                                                ПОЛ  0.0219
 14                                          DEBTS_MIN 

In [140]:
overall_results

{'r2 score for SVM on 3rd semester': [-0.6247849812066746,
  -0.3578848735439202,
  -0.39886549305083774,
  -0.5008526622702858,
  -0.2733832501315463],
 'MAE for SVM on 3rd semester': [1.281242924560125,
  1.207840426250829,
  1.130186855901932,
  1.259731554800866,
  1.1924680350400674],
 'r2 score for CatBoost on 3rd semester': [0.24444324501931214,
  0.13344523359439608,
  0.06258589090522815,
  0.2014247216068692,
  0.16501461899229308],
 'MAE for CatBoost on 3rd semester': [0.9296437831816454,
  0.9744407113105902,
  0.9503187702700896,
  0.9440069168446382,
  0.9551174252313113],
 'ROC-AUC for Random Forest on 3rd semester': [0.7443625854770433,
  0.7529977603994567,
  0.7275200358056981,
  0.7331445022288261,
  0.7283539008193501],
 'Accuracy for Random Forest on 3rd semester': [0.9443651925820257,
  0.920114122681883,
  0.9614835948644793,
  0.956490727532097,
  0.9379457917261056]}

In [30]:
rf_fitted, acc = pipeline_model(randf, X, y, 'debt')
print(acc)

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
0.7537012113055181
